# Azure AI Agent service - Custom Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import BingCustomSearchTool, ListSortOrder
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 17-Jun-2025 08:58:00


## Settings

In [4]:
load_dotenv("azure.env")

True

In [5]:
BING_CUSTOM_CONN_ID = os.environ["BING_CUSTOM_CONN_ID"]

In [6]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

## Initialize Bing Custom Search tool with connection id and instance name

In [7]:
instance_name = "travel"

# Initialize Bing Custom Search tool with connection id and instance name
bing_custom_tool = BingCustomSearchTool(connection_id=BING_CUSTOM_CONN_ID,
                                        instance_name=instance_name)

Exemple of instance definition:

<img src="custom.jpg">

## Agent creation

In [9]:
agent = project_client.create_agent(
    model='gpt-4o',
    name="travel-planner-agent",
    instructions="You are a travel planner dedicated to helping visitors discover top attractions, find the best hotels and restaurants, explore must-see sights, book flight tickets, and navigate public transportation with ease.",
    tools=bing_custom_tool.definitions,
    description="Travel planner",
    temperature=0.7,
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_eZaHkAqZBM8zHA2DQQpXM1y7


## Examples

In [10]:
prompt="List me the top 10 things to visit in Milan according to tripadvisor"

In [11]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_B8xn4IOBgETBiztH43FqakuL
💬 [2] Created message | ID: msg_7603bbrdN1dsIIiqmkiczCKb
🏃 [3] Created run     | ID: run_hJwctFuKiKyR9FhKvH0CHhfD
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_fQNYqkONrC3qf2kOeH7bZTQq, Type: message_creation
Details step: {'id': 'step_fQNYqkONrC3qf2kOeH7bZTQq', 'object': 'thread.run.step', 'created_at': 1750150706, 'run_id': 'run_hJwctFuKiKyR9FhKvH0CHhfD', 'assistant_id': 'asst_eZaHkAqZBM8zHA2DQQpXM1y7', 'thread_id': 'thread_B8xn4IOBgETBiztH43FqakuL', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150709, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_AvRx1iwldGVjoSJyR64jKLGC'}}, 'usage': {'prompt_tokens': 8176, 'completion_tokens': 297, 'total_tokens': 8473, 'prompt_token

In [17]:
prompt="What are the best hotels in Milan according to social networks?"

In [18]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")
    
# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_3cRtrg2rzaBuzpZtZUujSrO3
💬 [2] Created message | ID: msg_qwDBWjhTRlw7Kbd9l3RzUYbs
🏃 [3] Created run     | ID: run_7V1pde5xKiVBOiqvWnUn4BDs
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_fcS7DRYZCKQNFPUxh2epNv5R, Type: message_creation
Details step: {'id': 'step_fcS7DRYZCKQNFPUxh2epNv5R', 'object': 'thread.run.step', 'created_at': 1750150750, 'run_id': 'run_7V1pde5xKiVBOiqvWnUn4BDs', 'assistant_id': 'asst_eZaHkAqZBM8zHA2DQQpXM1y7', 'thread_id': 'thread_3cRtrg2rzaBuzpZtZUujSrO3', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150752, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_aDXx5LnN1G8c4brHDCgQPFYe'}}, 'usage': {'prompt_tokens': 4489, 'completion_tokens': 165, 'total_tokens': 4654, 'prompt_token_details': {'cached_to

In [14]:
prompt = "Find some flights from Paris to Milan"

In [15]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_M0uHWNKLDlCAml89T7zurSpv
💬 [2] Created message | ID: msg_qkP2vgzkqygFiDliJXZvLVEw
🏃 [3] Created run     | ID: run_Qbffmu94tLc0c13BW2I2tPi3
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_035PvJwaxcu6fWutSn5LcBah, Type: message_creation
Details step: {'id': 'step_035PvJwaxcu6fWutSn5LcBah', 'object': 'thread.run.step', 'created_at': 1750150723, 'run_id': 'run_Qbffmu94tLc0c13BW2I2tPi3', 'assistant_id': 'asst_eZaHkAqZBM8zHA2DQQpXM1y7', 'thread_id': 'thread_M0uHWNKLDlCAml89T7zurSpv', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150725, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_M78TYhgP4zvfgIh82BrO4qQe'}}, 'usage': {'prompt_tokens': 3825, 'completion_tokens': 158, 'total_tokens': 3983, 'prompt_token_details': {'cached_to

## Post processing

In [16]:
print(
    f"Agent Details:\n"
    f"📛 Name        : {agent.name}\n"
    f"🆔 ID          : {agent.id}\n"
    f"🧠 Model       : {agent.model}\n"
    f"📜 Instructions: {agent.instructions}"
)

Agent Details:
📛 Name        : travel-planner-agent
🆔 ID          : asst_eZaHkAqZBM8zHA2DQQpXM1y7
🧠 Model       : gpt-4o
📜 Instructions: You are a travel planner dedicated to helping visitors discover top attractions, find the best hotels and restaurants, explore must-see sights, book flight tickets, and navigate public transportation with ease.


In [ ]:
project_client.delete_agent(agent.id)
print(f"Deleted agent, agent ID: {agent.id}")